In [7]:
from tabela_de_estados import TabelaDeEstados
from Tabela_de_Simbolos import Tabela_de_Simbolos
from Token import Token

class Scanner:
  def __init__(self):
    self.numero_da_coluna = 0
    self.numero_da_linha = 0
    self.lexema = ''
    self.arrayParaIdentificarColuna = []


  def scanner(self, tabelaEstados: TabelaDeEstados, arrayDeCaracteres: str, tabelaDeSimbolos: Tabela_de_Simbolos):
    char = arrayDeCaracteres
    entrada = tabelaEstados.verificaTipoCaractere(char).strip()
    if(entrada == '$'):
      if (tabelaEstados.verificarSeEstaEmEstadoFinal()):
        newToken = Token(tabelaEstados.retornaClasse(), self.lexema, tabelaEstados.retornaTipo())
        return {'mensagem': 'TOKEN', 'token': newToken}
      return {'mensagem': 'EOF', 'token': Token(classe='EOF', lexema='EOF',tipo='NULO')}
    if (tabelaEstados.verificarSeEntradaPertenceAoAlfabeto(entrada)):
      if(tabelaEstados.verificarSeProximoEstadoEValido(entrada)):
        self.lexema = self.lexema + char.strip()
        return {'mensagem': 'chamar_novamente', 'token': None}
      elif(tabelaEstados.verificarSeEFinalDefinitivo(entrada)):
          newToken = Token(tabelaEstados.retornaClasse(), self.lexema, tabelaEstados.retornaTipo())
          return {'mensagem': 'TOKEN_DIRETO', 'token': newToken}
      elif (tabelaEstados.verificarSeEstaEmEstadoFinal()):
        newToken = Token(tabelaEstados.retornaClasse(), self.lexema, tabelaEstados.retornaTipo())
        return {'mensagem': 'TOKEN', 'token': newToken}
      else:
        tabelaEstados.lancarErro(self.numero_da_linha, self.numero_da_coluna)
        return {'mensagem': None, 'token': None}
    elif(tabelaEstados.entradaVazia(char)):
      if (tabelaEstados.verificarSeEstaEmEstadoFinal()):
        newToken = Token(tabelaEstados.retornaClasse(), self.lexema, tabelaEstados.retornaTipo())
        return {'mensagem': 'TOKEN', 'token': newToken}
      elif(tabelaEstados.verificarSeComentarioOuLiteral()):
        return {'mensagem': 'tratar_literal_comentario', 'token': None}

      else:
        return {'mensagem': 'chamar_novamente', 'token': None}
    else:
      self.numero_da_coluna = self.arrayParaIdentificarColuna.index(char) + 1
      print('ERRO LÉXICO – Caracter inválido, linha {}, coluna {}'.format(self.numero_da_linha, self.numero_da_coluna))
      return {'mensagem': None, 'token': None}



  def tratarFinalDaInstrucao(self, tabelaEstados: TabelaDeEstados):
    if (tabelaEstados.verificarSeEstaEmEstadoFinal()):
      newToken = Token(tabelaEstados.retornaClasse(), self.lexema, tabelaEstados.retornaTipo())
      return {'mensagem': 'TOKEN', 'token': newToken}
    else:
      tabelaEstados.lancarErro(self.numero_da_linha, self.numero_da_coluna)
      self.lexema = ''
      return {'mensagem': None, 'token': None}




  def limpa_codigo(self, codigoFonteFormatado):
    codigo_final = []
    for line in codigoFonteFormatado:
        for caractere in line:
            codigo_final.append(caractere)
    return codigo_final

  def scannerMain(self, tabelaEstados: TabelaDeEstados, codigoFonte, tabelaDeSimbolos: Tabela_de_Simbolos):
    for linha in codigoFonte:

      arrayDeCaracteres = self.limpa_codigo(linha)
      self.arrayParaIdentificarColuna = arrayDeCaracteres.copy()

      self.numero_da_linha += 1
      coluna = 0
      retorno = self.scanner(tabelaEstados, arrayDeCaracteres[0].replace(" ", ""), tabelaDeSimbolos)
      arrayDeCaracteres.pop(0)

      print('---------------------------------- LINHA {} ----------------------------------'.format(self.numero_da_linha))
      if(retorno['mensagem'] == 'EOF'):
        print('TOKEN - Classe: EOF, Lexema: EOF, Tipo: EOF')
        return # Final do arquivo
      while(retorno['mensagem'] != 'EOF' and len(arrayDeCaracteres) != 0):
        if(retorno['mensagem'] == 'chamar_novamente'):
          coluna += 1
          retorno = self.scanner(tabelaEstados, arrayDeCaracteres[0].replace(" ", ""), tabelaDeSimbolos)
          if(retorno['mensagem'] != 'TOKEN' and retorno['mensagem'] != 'TOKEN_DIRETO'):
            arrayDeCaracteres.pop(0)
          continue
        elif(retorno['mensagem'] == 'TOKEN'):
          tokenTabela = tabelaDeSimbolos.construir_token(tabelaEstados, retorno['token'].lexema)
          print('TOKEN - Classe: {}, Lexema: {}, Tipo: {}'.format(tokenTabela.classe, tokenTabela.lexema, tokenTabela.tipo))
          tabelaEstados.estado_atual = 0
          self.lexema = ''
          retorno = self.scanner(tabelaEstados, arrayDeCaracteres[0].replace(" ", ""), tabelaDeSimbolos)
          arrayDeCaracteres.pop(0)
          continue
        elif(retorno['mensagem'] == 'TOKEN_DIRETO'):
          tokenTabela = tabelaDeSimbolos.construir_token(tabelaEstados, retorno['token'].lexema)
          print('TOKEN - Classe: {}, Lexema: {}, Tipo: {}'.format(tokenTabela.classe, tokenTabela.lexema, tokenTabela.tipo))

          tabelaEstados.estado_atual = 0
          arrayDeCaracteres.pop(0)
          self.lexema = ''
          
          retorno = self.scanner(tabelaEstados, arrayDeCaracteres[0].replace(" ", ""), tabelaDeSimbolos)
          continue
        elif(retorno['mensagem']  == None): #Espaço vazio
          ultimoTokenAntesDoErro = self.tratarFinalDaInstrucao(tabelaEstados)
          if(ultimoTokenAntesDoErro['mensagem'] == 'TOKEN'):
            tokenTabela = tabelaDeSimbolos.construir_token(tabelaEstados, ultimoTokenAntesDoErro['token'].lexema)
            print('TOKEN - Classe: {}, Lexema: {}, Tipo: {}'.format(tokenTabela.classe, tokenTabela.lexema, tokenTabela.tipo))
          
          tabelaEstados.estado_atual = 0
          self.lexema = ''
          retorno['mensagem'] = 'chamar_novamente'
          coluna += 1
          continue

        elif(retorno['mensagem']  == 'tratar_literal_comentario'):
          self.lexema =  self.lexema + ' '
          retorno['mensagem'] = 'chamar_novamente'
          continue


      ultimoTokenDaLinha = self.tratarFinalDaInstrucao(tabelaEstados)
      if(ultimoTokenDaLinha['mensagem'] == 'TOKEN'):
        tokenTabela = tabelaDeSimbolos.construir_token(tabelaEstados, ultimoTokenDaLinha['token'].lexema)
        print('TOKEN - Classe: {}, Lexema: {}, Tipo: {}'.format(tokenTabela.classe, tokenTabela.lexema, tokenTabela.tipo))
        tabelaEstados.estado_atual = 0
        self.lexema = ''



      




    

In [8]:
arquivo = open("teste.txt", "r+", encoding="utf-8")
codigoFonte = arquivo.readlines()

def limpa_codigo(codigoFonte):
  novo_codigo = []
  ultimo_elemento = codigoFonte[-1]
  for i in codigoFonte:
    if(i.strip() == ultimo_elemento.strip()):
      i = i + '\n'
    nova_string = bytes(i, "utf-8").decode("unicode_escape")
    novo_codigo.append(nova_string[:][:-1])
  novo_codigo.append('$')
  return novo_codigo
  
codigoFormatado = limpa_codigo(codigoFonte)




codigoFormatado

['A;', 'B;', 'A;', '$\n', '$']

In [9]:
tabelaDeEstados = TabelaDeEstados()
tabelaDeSimbolos = Tabela_de_Simbolos()
# tabelaDeEstados.entradaVazia('     s     ')
teste = Scanner()

# array = ['a+b;', 'c<-102;' , '{txt;', 'a', '$']
# array = ['"ta"', "{a" ,'$', 'b']
# array = ['{teste;' ,'$']
# array = ['"sasa', "b" ,'$']
# array = [' ','abc','$']
# array = ['a+b' , 'c+ç{', '{','$']
# array = ['a+b;', 'c<-10e2' ,'$']
# array = ['{abc}', 'va','$']
# array = ['inteiro B, C','$']
# array = ['c<-102; {abc}','$']
array = ['teçste','$']
# teste.limpa_codigo(teste.codigoFontePalavra, array)
teste.scannerMain(tabelaDeEstados, codigoFormatado, tabelaDeSimbolos)

# token = Token('id', 'mamaqui', None)
# token2 = Token('id', 'token2', None)
# tabelaDeSimbolos.inserir_token(token)
# tabelaDeSimbolos.inserir_token(token2)
# busca = tabelaDeSimbolos.buscar_token(token)
# tabelaDeSimbolos.atualizar_token(busca)
# tabelaDeEstados.estado_atual = 11
# tabelaDeSimbolos.construir_token(tabelaDeEstados, 'mamaqui')
# tabelaDeSimbolos.construir_token(tabelaDeEstados, 'mamaqui2')
# print('-'*50)
# tabelaDeSimbolos.construir_token(tabelaDeEstados, 'mamaqui')
tabelaDeSimbolos.imprimir_tabela()

---------------------------------- LINHA 1 ----------------------------------
TOKEN ENCONTRADO:  None
TOKEN - Classe: id, Lexema: A, Tipo: NULO
TOKEN - Classe: PT_V, Lexema: ;, Tipo: NULO
---------------------------------- LINHA 2 ----------------------------------
TOKEN ENCONTRADO:  None
TOKEN - Classe: id, Lexema: B, Tipo: NULO
TOKEN - Classe: PT_V, Lexema: ;, Tipo: NULO
---------------------------------- LINHA 3 ----------------------------------
TOKEN ENCONTRADO:  Classe: id, Lexema: A, Tipo: NULO
TOKEN - Classe: id, Lexema: A, Tipo: NULO
TOKEN - Classe: PT_V, Lexema: ;, Tipo: NULO
---------------------------------- LINHA 4 ----------------------------------
TOKEN - Classe: EOF, Lexema: EOF, Tipo: EOF
Classe:inicio, Lexema:inicio, Tipo:inicio
Classe:varinicio, Lexema:varinicio, Tipo:varinicio
Classe:varfim, Lexema:varfim, Tipo:varfim
Classe:escreva, Lexema:escreva, Tipo:escreva
Classe:leia, Lexema:leia, Tipo:leia
Classe:se, Lexema:se, Tipo:se
Classe:entao, Lexema:entao, Tipo:entao


In [10]:
from Token import Token
from tabela_de_estados import TabelaDeEstados

class Tabela_de_Simbolos:
    def __init__(self):

        self.tabela = []

        self.tabela.append(Token(classe="inicio",lexema="inicio",tipo="inicio"))
        self.tabela.append(Token(classe="varinicio",lexema="varinicio",tipo="varinicio"))
        self.tabela.append(Token(classe="varfim", lexema="varfim", tipo="varfim"))
        self.tabela.append(Token(classe="escreva",lexema="escreva", tipo="escreva"))
        self.tabela.append(Token(classe="leia", lexema="leia",tipo= "leia"))
        self.tabela.append(Token(classe="se", lexema="se",tipo= "se"))
        self.tabela.append(Token(classe="entao",lexema= "entao",tipo= "entao"))
        self.tabela.append(Token (classe="fimse",lexema= "fimse",tipo= "fimse"))
        self.tabela.append(Token(classe="fim",lexema= "fim", tipo="fim"))
        self.tabela.append(Token(classe="inteiro",lexema= "inteiro",tipo= "inteiro"))
        self.tabela.append(Token(classe="literal",lexema= "literal",tipo= "literal"))
        self.tabela.append(Token(classe="real",lexema= "real",tipo= "real"))


    def buscar_token(self, token:Token):
            for t in self.tabela:
                if(t.lexema == token.lexema):
                    return t
            return None
        

    def inserir_token(self, token:Token):
            token_encontrado = self.buscar_token(token)
            if(token_encontrado is None):
                if(token.classe == "id"):
                    self.tabela.append(token)
                return token
            else:
                return token_encontrado

    def atualizar_token(self, token: Token):
        if(token.classe == "id"):
                token_tabela_simbolos = self.buscar_token(token)
                if(token_tabela_simbolos):
                    self.tabela.remove(token_tabela_simbolos)
                    self.tabela.append(token)  

    def imprimir_tabela(self):
            for t in self.tabela:
                print(f"Classe:{t.classe}, Lexema:{t.lexema}, Tipo:{t.tipo}")

    def construir_token(self, tabela_de_estados: TabelaDeEstados, lexema: str):
        classe = tabela_de_estados.retornaClasse()
        tipo = tabela_de_estados.retornaTipo()
        if classe == 'id':
            token = self.buscar_token(Token(classe, lexema, tipo))
            if token is None:
                token = self.inserir_token(Token(classe, lexema, tipo))

            return token
        else:
            return Token(classe, lexema, tipo)



tabelaDeEstados = TabelaDeEstados()
tabelaDeSimbolos = Tabela_de_Simbolos()


tabelaDeEstados.estado_atual = 28
# token = tabelaDeSimbolos.construir_token(tabelaDeEstados, 'fimse')
newToken = Token('id', 'dsdsd', 'NULO')
# token = tabelaDeSimbolos.construir_token(newToken)
token = tabelaDeSimbolos.construir_token(tabelaDeEstados, 'erer')
tabela = tabelaDeSimbolos.imprimir_tabela()

# print('AAAAAAAAAAAA: ', token)
# print('TABELA:: ', tabela)

Classe:inicio, Lexema:inicio, Tipo:inicio
Classe:varinicio, Lexema:varinicio, Tipo:varinicio
Classe:varfim, Lexema:varfim, Tipo:varfim
Classe:escreva, Lexema:escreva, Tipo:escreva
Classe:leia, Lexema:leia, Tipo:leia
Classe:se, Lexema:se, Tipo:se
Classe:entao, Lexema:entao, Tipo:entao
Classe:fimse, Lexema:fimse, Tipo:fimse
Classe:fim, Lexema:fim, Tipo:fim
Classe:inteiro, Lexema:inteiro, Tipo:inteiro
Classe:literal, Lexema:literal, Tipo:literal
Classe:real, Lexema:real, Tipo:real
